<a href="https://colab.research.google.com/github/Kkuntal990/CTW_2020/blob/master/Siamese2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}") ; print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}") ; print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.33GB
Used: 832.95MB
Percentage: 4.6%


In [2]:
! nvidia-smi

Thu Jul  9 15:56:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
import h5py
f = h5py.File("/content/gdrive/My Drive/CTW2020/Processed Data/udata2020.hdf5","r")
import numpy as np
_ , key = next(enumerate(f.keys()))
X = f[key][:]
f.close()

In [5]:
X.shape

(36192, 56, 924, 2)

In [6]:
X = np.sqrt(X[:,:,:,0]**2 + X[:,:,:,1]**2)

In [7]:
import h5py
f = h5py.File("/content/gdrive/My Drive/CTW2020/Processed Data/Labelled_1.hdf5","r")

H = f["H_Est"][:]
Pos = f["Pos"][:]
SNR = f["SNR"][:]
f.close()

print(H.shape)
print(SNR.shape)

(4979, 56, 924, 2)
(4979, 56)


In [8]:
H = np.sqrt(H[:,:,:,0]**2 + H[:,:,:,1]**2)

In [9]:
from sklearn.model_selection import train_test_split
H_Train, H_Test , Pos_Train , Pos_Test = train_test_split(H,Pos,test_size=0.1, random_state=42)

In [10]:
import torchvision
import torch
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import os
from torchvision.utils import save_image
from torch.autograd import Variable

In [11]:
H_Train, H_Val , Pos_Train , Pos_Val = train_test_split(H_Train,Pos_Train,test_size=0.1, random_state=99)

In [12]:
print(H_Train.shape, H_Val.shape, H_Test.shape)


(4032, 56, 924) (449, 56, 924) (498, 56, 924)


In [13]:
H_Train = torch.tensor(H_Train,dtype=torch.float)
Pos_Train = torch.tensor(Pos_Train,dtype=torch.float)
H_Val = torch.tensor(H_Val,dtype=torch.float)
Pos_Val = torch.tensor(Pos_Val,dtype=torch.float)

In [14]:
class Sammon(nn.Module):
    def __init__(self):
        super(Sammon, self).__init__()
        
        
        self.main = nn.Sequential(
            nn.Conv2d(1,2, (2,4), stride=(1,2)),
            nn.ReLU(),
            nn.Conv2d(2, 4, (2,4), stride=(1,4)),
            nn.ReLU())
        
        self.Project = nn.Sequential(nn.Linear(49680,20000),
                                     nn.Tanh(),
                                     nn.Linear(20000,9000),
                                     nn.Softplus(),
                                     nn.Linear(9000,4500),
                                     nn.Tanh(),)
                                     
          
            
    def forward(self, input):
        return self.Project(self.main(input).view(-1,49680))


class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()        
        self.Project = nn.Sequential(nn.Linear(4500,2048),
                                     nn.Tanh(),
                                     nn.Linear(2048,1024),
                                     nn.Softplus(),
                                     nn.Linear(1024,512),
                                     nn.Tanh(),
                                     nn.Linear(512,256),
                                     nn.Tanh(),
                                     nn.Linear(256,128),
                                     nn.Tanh(),
                                     nn.Linear(128,64),
                                     nn.Tanh(),
                                     nn.Linear(64,32),
                                     nn.Softplus(),
                                     nn.Linear(32,16),
                                     nn.Tanh(),
                                     nn.Linear(16,8),
                                     nn.Tanh(),
                                     nn.Linear(8,3))
          
            
    def forward(self, input):
        return self.Project(input).view(-1,3)


In [15]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
sammon = Sammon().to(device)
reg = Regressor().to(device)

In [16]:
opt1 = optim.Adam(sammon.parameters(), lr=1e-4, betas=(0.5, 0.999))
opt2 = optim.Adam(reg.parameters(), lr=1e-4, betas=(0.5, 0.999))
MSE = nn.MSELoss()

In [17]:
summary(sammon, (1, 56,924))
summary(reg, (4500,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 2, 55, 461]              18
              ReLU-2           [-1, 2, 55, 461]               0
            Conv2d-3           [-1, 4, 54, 115]              68
              ReLU-4           [-1, 4, 54, 115]               0
            Linear-5                [-1, 20000]     993,620,000
              Tanh-6                [-1, 20000]               0
            Linear-7                 [-1, 9000]     180,009,000
          Softplus-8                 [-1, 9000]               0
            Linear-9                 [-1, 4500]      40,504,500
             Tanh-10                 [-1, 4500]               0
Total params: 1,214,133,586
Trainable params: 1,214,133,586
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.20
Forward/backward pass size (MB): 1.66
Params size (MB): 4631

In [18]:
A = list(range(len(X)))*4
pairs = []
import random
while len(A)>1:
  t = random.randint(1,len(A)-1)
  if(A[t]!=A[0]):
    pairs.append((A[0],A[t]))
    del A[0]
    del A[t-1]


In [19]:
pairs = np.array(pairs)

In [20]:
pairs.shape

(72384, 2)

In [21]:
N_b2 = len(H_Train)
N_b = len(pairs)
N_val = len(H_Val)
bs1= 64
bs2 = 8
epsilon = 1e-6

print(N_b2, N_b, N_val)

4032 72384 449


In [24]:
X.shape

(36192, 56, 924)

In [22]:
def custom_loss(XX, YY):
  return torch.sum((1/XX)*(torch.square(XX-YY)))



In [23]:
for i in range(100):
  batch1=0
  batch2=0
  for j in range(0,N_b,bs1):
    sammon.zero_grad()
    end = min(j+bs1,N_b)
    Xn,Xm = torch.tensor(X[pairs[j:end,0]],dtype=torch.float),  torch.tensor(X[pairs[j:end,1]],dtype=torch.float)
    Xn , Xm = Variable(Xn, requires_grad=True) , Variable(Xm, requires_grad=True)
    Xn = Xn.to(device)
    Xm = Xm.to(device)
    Yn = sammon(Xn.view(-1,1,56,924))
    Ym = sammon(Xm.view(-1,1,56,924))
    print(Xn.view(-1,1,56,924).shape, Yn.shape)

    XX = torch.norm((Xn-Xm),p=2, dim = [1,2])
    YY = torch.norm((Yn-Ym),p=2, dim = 1)
    XX = torch.sqrt(torch.square(XX) + epsilon)
    YY = torch.sqrt(torch.square(YY) + epsilon)
    loss = custom_loss(XX, YY)
    #print(loss)
    loss.backward()
    opt1.step()
    batch1+=1
    if(batch1%10 == 0):
      print("Batch",batch1,"/",(N_b//bs1)+1,"Sammon Loss :" , loss.item())

  for k in range(0,N_b2,bs2):
    end = min(k+bs2,N_b2)
    H1 = H_Train[k:end]
    sammon.zero_grad()
    reg.zero_grad()
    H1 = Variable(H1,requires_grad=True)
    H1 = H1.to(device)
    Y1 = reg(sammon(H1.view(-1,1,56,924)))
    loss_1 = MSE(Y1,Pos_Train[k:end].to(device)).view(-1)
    loss_1.backward()
    opt1.step()
    opt2.step()
    batch2+=1
    if(batch2%10 == 0):
      print("Batch",batch2,"/",(N_b2//bs2)+1,"Regressor Loss :" , loss_1.item())
  print("Epoch ",i,"/",100," Sammon Loss :",loss.item() ," Regressor Loss :" , loss_1.item())
  for l in range(N_val):
    H_val = torch.tensor(H_Val[l],dtype=torch.float)
    H_val = H_val.to(device)
    with torch.no_grad():
      Y_Val = reg(sammon(H_val.view(-1,1,56,924)))
      loss_Val = MSE(Y_Val,Pos_Val[l].to(device)).view(-1)
  print("Val Loss :" , loss_Val.item())

    

      
  


torch.Size([64, 1, 56, 924]) torch.Size([32, 4500])


RuntimeError: ignored

In [ ]:
pairs[2:9,0]

array([2, 3, 4, 5, 6, 7, 8])